# Equivalent Circuit Parameter Identification

## Estimating resistance and capacitance values from HPPC data

This notebook demonstrates how to identify stationary parameters for a Thevenin model. The Thevenin model is an empirical circuit model capable of capturing the electrical response of a battery. This model can be extended with a thermal submodel, as well as additional parallel resistor-capacitor (RC) branches.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by upgrading some dependencies and importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
%pip install --upgrade pandas -q

import numpy as np
import pandas as pd
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.
/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


## Importing parameters

First let's create the model with two RC elements.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

Parameters can be defined by importing a JSON representation, such as the one in the PyBOP examples. To import via JSON, either download the example file, or create your own and update the path below to reference the corresponding file.

In [ ]:
import json

json_path = "../../parameters/initial_ecm_parameters.json"
with open(json_path) as f:
    params_dict = json.load(f)

parameter_values = pybamm.ParameterValues(params_dict)
parameter_values.update(
    {
        "Open-circuit voltage [V]": model.default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    },
    check_already_exists=False,
)

Alternatively, define the initial parameter set with a dictionary. Ensure you have definitions for all R's, C's, and initial overpotentials for any additional RC elements.

In this example, we use the default parameter value for the "Open-circuit voltage [V]" as provided by the original PyBaMM class. To update this, provide a function definition that matches this [function](https://github.com/pybamm-team/PyBaMM/blob/1943aa5ab2895b5378220595923dbae3d66b13c9/pybamm/input/parameters/ecm/example_set.py#L17).

In [ ]:
parameter_values = model.default_parameter_values
parameter_values.update(
    {
        "Cell capacity [A.h]": 3,
        "Nominal cell capacity [A.h]": 3,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "R2 [Ohm]": 0.002,
        "C2 [F]": 3e4,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

## Importing data

We will use experimental data for a single HPPC pulse from an open dataset [1]. This is imported and used to construct the `pybop.Dataset` class.

[1] Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1

In [ ]:
file_loc = r"../../data/Samsung_INR21700/sample_hppc_pulse.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)
parameter_values.set_initial_state(f"{dataset['Voltage [V]'][0]} V");

## Identifying the parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. In this example, we've construct a two-branch Thevenin model, so we will select all five resistance and capacitance parameters for identification. This isn't recommended for real-life application as it is difficult to guarantee identifiablity with this large a parameter space. The initial guess for each resistance parameter is generated from a random sample of the prior distributions. These are influenced by the `r_guess` parameter below.

In [ ]:
r_guess = 0.005
parameter_values.update(
    {
        "R0 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r_guess, r_guess / 10),
            bounds=[0, 0.2],
        ),
        "R1 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r_guess, r_guess / 10),
            bounds=[0, 0.2],
        ),
        "C1 [F]": pybop.Parameter(
            prior=pybop.Gaussian(500, 100),
            bounds=[100, 10000],
        ),
        "R2 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r_guess, r_guess / 10),
            bounds=[0, 0.2],
        ),
        "C2 [F]": pybop.Parameter(
            prior=pybop.Gaussian(2000, 500),
            bounds=[100, 10000],
        ),
    }
)

The `Problem` class provides us with a single class that holds the objects we need to evaluate our selected `SumSquaredError` cost function.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)

Next, we construct the optimisation class with our algorithm of choice and run it. For the sake of reducing the runtime of this example, we limit the maximum iterations; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=20,
    max_iterations=100,
)
optim = pybop.XNES(problem, options=options)
result = optim.run()

## Plotting and visualisation

Next, we use PyBOP's plotting utilities to visualise the results of the optimisation. This provides us with a visual confirmation of the optimiser's converged parameter values in the time-domain output.

In [ ]:
pybop.plot.problem(problem, inputs=result.best_inputs, title="Optimised Comparison");

### Convergence and parameter trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
result.plot_convergence()
result.plot_parameters();

## Using the estimated parameter for thermal predictions

With the estimated RC parameters, the temperature distribution for a given drive cycle can be calculated using the identified Thevenin model. We use a `.xlsx` file containing time-series current data as a `pybamm.Experiment`. A sample file is used here, but user's may choose to upload customized drive cycle.

In [ ]:
file_loc = r"../../data/Samsung_INR21700/sample_drive_cycle.xlsx"
df = pd.read_excel(file_loc, sheet_name="Sheet3", index_col=None, na_values=["NA"])

# Remove duplicate rows, keeping the first occurrence
df = df.drop_duplicates(subset=["Time"], keep="first")

# Create the pybamm experiment
experiment = pybamm.Experiment([pybamm.step.current(df.to_numpy())])

experiment.steps

[Step([[   0    3]
  [   1    3]
  [   2    3]
  ...
  [3598    3]
  [3599    3]
  [3600    3]], duration=3600)]

Update the estimated RC values. These values will be used to calculate heat generation and corresponding temperature distribution in the thermal submodel. Visualisation of voltage response and cell temperature is plotted below using the PyBaMM solution.

In [ ]:
parameter_values.update(result.best_inputs)
sol = pybamm.Simulation(
    model, parameter_values=parameter_values, experiment=experiment
).solve(initial_soc=0.95)
sol.plot();

## Concluding thoughts

This notebook illustrates how to extract EC parameters from an HPPC pulse using PyBOP, providing insights into the optimisation process through various visualisations. The estimated parameters are then used to run a thermal submodel.